# Guards Augment Pattern Matching
[Guards in Hexdocs](https://hexdocs.pm/elixir/master/guards.html)

[Full list of guards in Kernel Hexdocs](https://hexdocs.pm/elixir/master/Kernel.html#guards)

This is not meant to be an exhaustive introduction to guards. It is a brief intro to using guards with pattern matching.

## List of allowed expressions

* comparison operators (`==`, `!=`, `===`, `!==`, `>`, `>=`, `<`, `<=`)
* strictly boolean operators (`and`, `or`, `not`). Note `&&`, `||`, and `!` sibling operators are not allowed as they're not strictly boolean - meaning they don't require arguments to be booleans
* arithmetic unary and binary operators (`+`, `-`, `+`, `-`, `*`, `/`)
* `in` and `not` in operators (as long as the right-hand side is a list or a range)
* "type-check" functions (`is_atom/1`, `is_binary/1`, `is_bitstring/1`, `is_boolean/1`, `is_float/1`, `is_function/1`, `is_function/2`, `is_integer/1`, `is_list/1`, `is_map/1`, `is_nil/1`, `is_number/1`, `is_pid/1`, `is_port/1`, `is_reference/1`)
* functions that work on built-in datatypes (`abs/1`, `binary_part/3`, `bit_size/1`, `byte_size/1`, `ceil/1`, `div/2`, `elem/2`, `floor/1`, `hd/1`, `length/1`, `map_size/1`, `node/0`, `node/1`, `rem/2`, `round/1`, `self/0`, `tl/1`, `trunc/1`, `tuple_size/1`)


---
Guards work in `case` statements

In [2]:
x = 3
case x do
  1 -> :one
  2 -> :two
  n when is_integer(n) and n > 2 -> :larger_than_two
end

:larger_than_two

---
Failing guards are silent, they just prevent the clause from matching.

In [3]:
length("hello")

ArgumentError: 1

In [3]:
case "hello" do
  something when length(something) > 0 ->
    :length_worked
  _anything_else ->
    :length_failed
end

  nofile:2



:length_failed

## Custom Guards

In [6]:
defmodule MyInteger do
  defguard is_even(value) when is_integer(value) and rem(value, 2) == 0
end

  nofile:1



{:module, MyInteger, <<70, 79, 82, 49, 0, 0, 6, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 208, 0, 0, 0, 22, 16, 69, 108, 105, 120, 105, 114, 46, 77, 121, 73, 110, 116, 101, 103, 101, 114, 8, 95, 95, 105, 110, 102, ...>>, {:is_even, 1}}

In [5]:
import MyInteger, only: [is_even: 1]

MyInteger

In [14]:
defmodule Mathy do

  def yell(number) when is_even(number) do
    "#{number} is EVEN!!!"
  end
  
  def yell(number)do
    "#{number} is ODD!!!"
  end
end

  nofile:1



{:module, Mathy, <<70, 79, 82, 49, 0, 0, 5, 164, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 158, 0, 0, 0, 16, 12, 69, 108, 105, 120, 105, 114, 46, 77, 97, 116, 104, 121, 8, 95, 95, 105, 110, 102, 111, 95, 95, 7, ...>>, {:yell, 1}}

In [8]:
import Mathy, only: [yell: 1]

Mathy

In [15]:
yell(1)

"1 is ODD!!!"

In [16]:
yell(2)

"2 is EVEN!!!"

---
Multiple guards in the same clause

In [17]:
defmodule MultiGuard do
  
  def foo(term) when is_integer(term) or is_float(term) or is_nil(term), do: :maybe_number
  
  def foo(_other), do: :something_else
end

{:module, MultiGuard, <<70, 79, 82, 49, 0, 0, 4, 248, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 160, 0, 0, 0, 16, 17, 69, 108, 105, 120, 105, 114, 46, 77, 117, 108, 116, 105, 71, 117, 97, 114, 100, 8, 95, 95, 105, 110, ...>>, {:foo, 1}}

In [18]:
import MultiGuard, only: [foo: 1]

MultiGuard

In [19]:
foo(1)

:maybe_number

In [20]:
foo(:one)

:something_else

... can also be written as

In [21]:
defmodule MultiGuard2 do

  def foo2(term)
      when is_integer(term)
      when is_float(term)
      when is_nil(term) do
    :maybe_number
  end

  def foo2(_other) do
    :something_else
  end
end

{:module, MultiGuard2, <<70, 79, 82, 49, 0, 0, 4, 252, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 162, 0, 0, 0, 16, 18, 69, 108, 105, 120, 105, 114, 46, 77, 117, 108, 116, 105, 71, 117, 97, 114, 100, 50, 8, 95, 95, 105, ...>>, {:foo2, 1}}

In [22]:
import MultiGuard2, only: [foo2: 1]

MultiGuard2

In [23]:
foo2(1)

:maybe_number

In [24]:
foo2(:one)

:something_else

---
If any function in a guard raises, then the entire clause fails.

In [25]:
defmodule Check do
  # If given a tuple, map_size/1 will raise, and tuple_size/1 will not be evaluated
  def empty?(val) when map_size(val) == 0 or tuple_size(val) == 0, do: true
  def empty?(val) when map_size(val) == 0 
                  when tuple_size(val) == 0,do true
  
  def empty?(_val), do: false
end

{:module, Check, <<70, 79, 82, 49, 0, 0, 4, 208, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 146, 0, 0, 0, 16, 12, 69, 108, 105, 120, 105, 114, 46, 67, 104, 101, 99, 107, 8, 95, 95, 105, 110, 102, 111, 95, 95, 7, ...>>, {:empty?, 1}}

In [26]:
import Check, only: [empty?: 1]

Check

In [27]:
Check.empty?(%{})

true

In [29]:
Check.empty?({}) # true was expected!

false